In [27]:
import numpy as np
import pandas as pd
import math
import time
from xclib.data import data_utils

In [28]:
datapath = './../virus/ass3_parta_data/'
num_classes = 2

In [29]:
def read_datafile(filepath):
    temp = data_utils.read_sparse_file(filepath, force_header=True)
    return temp

In [30]:
train_data = read_datafile(datapath + 'train_x.txt')
X_train = train_data.todense()
y_train = np.genfromtxt(datapath + 'train_y.txt', dtype=float).reshape((-1,1))
print(X_train.shape)
print(y_train.shape)

(64713, 482)
(64713, 1)


In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

start = time.time()

clf = RandomForestClassifier(n_estimators=50, oob_score=True, max_features=0.1, min_samples_split=2, n_jobs=11, criterion='entropy')
# clf.fit(X_train, y_train.reshape(-1))

end = time.time()
print("Time taken = {} s".format(end-start))

Time taken = 7.653236389160156e-05 s


In [32]:
from sklearn.model_selection import GridSearchCV

start = time.time()

parameters = {
    'n_estimators': [50, 150, 250, 350, 450],
    'max_features': [0.1, 0.3, 0.5, 0.7, 0.9],
    'min_samples_split': [2, 4, 6, 8, 10],
}

# gscv = GridSearchCV(estimator=clf, param_grid=parameters, cv=3, n_jobs=1)
# gscv_fit = gscv.fit(X_train, y_train.reshape(-1))

end = time.time()
print("Time taken = {} s".format(end-start))

Time taken = 0.00042057037353515625 s


In [33]:
count = 1
best_score = 0.0
best_param = [0, 0.0, 0]
for n_est in parameters['n_estimators']:
    for max_feat in parameters['max_features']:
        for min_sam in parameters['min_samples_split']:

            start = time.time()
            clf = RandomForestClassifier(n_estimators=n_est, oob_score=True, max_features=max_feat, min_samples_split=min_sam, n_jobs=-1, criterion='entropy')
            clf.fit(X_train, y_train.reshape(-1))
            if clf.oob_score_ > best_score:
                best_score = clf.oob_score_
                best_param = [n_est, max_feat, min_sam]
            end = time.time()
            print("{}: OOB Score = {}".format(count, clf.oob_score_))
            print("\tTime taken = {} s".format(end-start))
            count += 1

1: OOB Score = 0.7977067977067978
	Time taken = 7.017951488494873 s
2: OOB Score = 0.8026671611577272
	Time taken = 7.016727924346924 s
3: OOB Score = 0.8040424644198229
	Time taken = 7.130366086959839 s
4: OOB Score = 0.8065303725681084
	Time taken = 7.228735685348511 s
5: OOB Score = 0.8070248636286372
	Time taken = 7.403018951416016 s
6: OOB Score = 0.7997002147945544
	Time taken = 17.058873176574707 s
7: OOB Score = 0.8012145936674239
	Time taken = 14.71429443359375 s
8: OOB Score = 0.8042433514131627
	Time taken = 16.023998260498047 s
9: OOB Score = 0.8057886359773152
	Time taken = 16.181668758392334 s
10: OOB Score = 0.8060822400445042
	Time taken = 16.01027536392212 s
11: OOB Score = 0.7998083847140451
	Time taken = 24.535414934158325 s
12: OOB Score = 0.800658291224329
	Time taken = 24.211551904678345 s
13: OOB Score = 0.8029298595336332
	Time taken = 24.042564153671265 s
14: OOB Score = 0.8049696351583144
	Time taken = 23.615413665771484 s
15: OOB Score = 0.8083074498168837
	T

In [45]:
start = time.time()
# best_params = gscv_fit.best_params_
# oclf = RandomForestClassifier(n_estimators=best_params["n_estimators"], oob_score=True, max_features=best_params["max_features"], min_samples_split=best_params["min_samples_split"], n_jobs=-1, criterion='entropy')
oclf = RandomForestClassifier(n_estimators=best_param[0], oob_score=True, max_features=best_param[1], min_samples_split=best_param[2], n_jobs=-1, criterion='entropy')
oclf.fit(X_train, y_train.reshape(-1))
end = time.time()
print("Time taken = {} s".format(end-start))

Time taken = 62.57508182525635 s


In [46]:
model = oclf

In [47]:
print(best_param)
print(best_score)

[450, 0.1, 10]
0.8109035278846599


In [49]:
# Making predictions
X_test = read_datafile(datapath + 'train_x.txt').toarray()
y_preds = model.predict(X_test)
y_test = np.genfromtxt(datapath + 'train_y.txt', dtype=float)
check = (y_preds == y_test)
accuracy = np.sum(check) * 100.0 / len(check)
print("Training accuracy = {}%".format(accuracy))
X_test = read_datafile(datapath + 'test_x.txt').toarray()
y_preds = model.predict(X_test)
y_test = np.genfromtxt(datapath + 'test_y.txt', dtype=float)
check = (y_preds == y_test)
accuracy = np.sum(check) * 100.0 / len(check)
print("Test accuracy = {}%".format(accuracy))
X_test = read_datafile(datapath + 'valid_x.txt').toarray()
y_preds = model.predict(X_test)
y_test = np.genfromtxt(datapath + 'valid_y.txt', dtype=float)
check = (y_preds == y_test)
accuracy = np.sum(check) * 100.0 / len(check)
print("Validation accuracy = {}%".format(accuracy))

Training accuracy = 87.59909137267628%
Test accuracy = 80.74729961522414%
Validation accuracy = 80.67865751900612%
